# 🔍 SHAP Interpretation for Tox21 Models (Random Forest)

This notebook generates SHAP summary plots for each of the 12 Tox21 toxicity targets using pre-trained Random Forest models.

Make sure that:
- Your model files are saved in the `final_models/` directory
- Your ECFP4 features are stored in the variable `X`
- Your target labels are part of the `df` DataFrame

In [7]:
import shap
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Create necessary directories
os.makedirs("final_models", exist_ok=True)
os.makedirs("shap_summaries", exist_ok=True)
os.makedirs("shap_summaries/rf", exist_ok=True)

# Load your dataset
df = pd.read_csv("data/tox21.csv")

# ECFP4 feature matrix must be defined as X
# Example: X = np.array([...]) generated from RDKit


In [10]:
from rdkit import Chem
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
import numpy as np

# Generate ECFP4 fingerprints (radius=2, 1024 bits)
morgan_gen = GetMorganGenerator(radius=2, fpSize=1024)

def smiles_to_ecfp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros((morgan_gen.GetNumBits(),))
    return np.array(morgan_gen.GetFingerprint(mol))

# Apply to all rows
X = np.array([smiles_to_ecfp(s) for s in df['smiles']])


[00:51:32] WARNING: not removing hydrogen atom without neighbors


In [11]:
# Target List

targets = [
    'NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 
    'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 
    'SR-HSE', 'SR-MMP', 'SR-p53'
]


In [12]:
for target in targets:
    print(f"🚀 Processing {target} (Random Forest)")

    y = df[target]
    mask = y.notna()
    X_clean = X[mask]
    y_clean = y[mask].values

    if len(np.unique(y_clean)) < 2:
        print(f"⚠️ Skipping {target} — only one class present.")
        continue

    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.2, stratify=y_clean, random_state=42
    )

    # Train Random Forest
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        class_weight='balanced',
        random_state=42
    )
    model.fit(X_train, y_train)

    # Save the model
    joblib.dump(model, f"final_models/{target}_Random_Forest.joblib")

    # SHAP summary plot
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)

    plt.figure()
    shap.summary_plot(shap_values[1], X_test, plot_type="bar", max_display=20, show=False)
    plt.title(f"{target} - SHAP Summary (Random Forest)")
    plt.savefig(f"shap_summaries/rf/{target}_Random_Forest_shap_summary.png", bbox_inches='tight')
    plt.close()
    print(f"✅ Saved model and SHAP plot for {target}")


🚀 Processing NR-AR (Random Forest)
✅ Saved model and SHAP plot for NR-AR
🚀 Processing NR-AR-LBD (Random Forest)
✅ Saved model and SHAP plot for NR-AR-LBD
🚀 Processing NR-AhR (Random Forest)
✅ Saved model and SHAP plot for NR-AhR
🚀 Processing NR-Aromatase (Random Forest)
✅ Saved model and SHAP plot for NR-Aromatase
🚀 Processing NR-ER (Random Forest)
✅ Saved model and SHAP plot for NR-ER
🚀 Processing NR-ER-LBD (Random Forest)
✅ Saved model and SHAP plot for NR-ER-LBD
🚀 Processing NR-PPAR-gamma (Random Forest)
✅ Saved model and SHAP plot for NR-PPAR-gamma
🚀 Processing SR-ARE (Random Forest)
✅ Saved model and SHAP plot for SR-ARE
🚀 Processing SR-ATAD5 (Random Forest)
✅ Saved model and SHAP plot for SR-ATAD5
🚀 Processing SR-HSE (Random Forest)
✅ Saved model and SHAP plot for SR-HSE
🚀 Processing SR-MMP (Random Forest)
✅ Saved model and SHAP plot for SR-MMP
🚀 Processing SR-p53 (Random Forest)
✅ Saved model and SHAP plot for SR-p53


The SHAP summary plots identify the top fingerprint features that influence the model’s prediction of toxicity. Each bar represents an ECFP4 bit (hashed molecular substructure) ranked by its average absolute SHAP value across all test samples. Features with longer bars have stronger contributions to the model’s decisions. While the fingerprint bits are not directly interpretable, their importance suggests consistent structure-activity relationships captured by the model.